In [1]:
import numpy 
import pandas

# GA with pymoo

In [9]:
# !pip install pymoo

In [1]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.factory import get_problem
from pymoo.optimize import minimize

problem = get_problem("g01")
print(problem)

algorithm = GA(
    pop_size=100,
    eliminate_duplicates=True)

res = minimize(problem,
               algorithm,
               seed=1,
               verbose=False)

print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

# name: G1
# n_var: 13
# n_obj: 1
# n_constr: 9

Best solution found: 
X = [1.         0.99999999 1.         0.99999998 0.99999996 1.
 0.99999992 0.99999999 1.         2.99906987 2.99525757 2.98473236
 0.99999999]
F = [-14.97905952]


# Example- 2 using NSGA2 pymoo

In [3]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.factory import get_problem
from pymoo.optimize import minimize
import numpy as np
import pandas as pd
import autograd.numpy as anp
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter

In [11]:
class MyProblem1(Problem):

    def __init__(self):
        super().__init__(n_var=100,
                         n_obj=3,
                         n_constr=0,
                         xl=anp.array([-1]*100),
                         xu=anp.array([2]*100))
  
    def _evaluate(self, x, out, *args, **kwargs):
        f1 = 1 - np.exp(-((x - 1 / np.sqrt(self.n_var)) ** 2).sum(axis=1))
        f2 = 1 - np.exp(-((x + 1 / np.sqrt(self.n_var)) ** 2).sum(axis=1))
        out["F"] = np.column_stack([f1, f2])
        out["G"] = np.column_stack([f1, f2])

In [15]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.factory import get_problem
from pymoo.optimize import minimize

print(MyProblem1())

algorithm = NSGA2(
    pop_size=1,
    eliminate_duplicates=True)

res = minimize(MyProblem1(),
               algorithm,
               seed=10,
               verbose=False)

print("Best solution found: %s" % res.X)
print("Function value: %s" % res.F)
print("Constraint violation: %s" % res.CV)

# name: MyProblem1
# n_var: 100
# n_obj: 3
# n_constr: 0

Best solution found: [[ 1.31396193 -0.86609636  0.90507575  1.24641165  0.49552104 -0.64670542
  -0.40581141  1.28159214 -0.49266749 -0.73498056  1.3765791   1.89447108
  -0.93017743  0.53657679  1.43786288  0.8375782   1.16526595  0.13624156
   1.75332237  1.14372735  0.6276331  -0.57348986  0.12002228  1.02240085
   0.32549952  0.31963353  0.92887474  0.53941473  0.95119155  0.80311686
   1.41566959  0.56494146  1.72594664 -0.04229173 -0.71538145 -0.01751594
  -0.65804691  1.47150864 -0.85931104  0.89847423  0.64275847  1.45786099
  -0.40315738  1.57055091  0.10450447  1.26394307 -0.11211488  1.65180944
  -0.02346509 -0.50495231  0.17758773 -0.78319108  1.46331697 -0.54654394
  -0.01053963  1.83278214  1.96287642  0.36891364  1.77547713 -0.19963396
   0.79211494  1.70849528  0.60367385  0.77060409 -0.8821547   0.07154528
  -0.76116073 -0.12213244 -0.00784206  1.29611671 -0.88012237  0.28847654
   0.00715813  0.90947343  0.0390

# Example- 3 using GA pymoo

In [16]:
class MyProblem3(Problem):

    def __init__(self):
        super().__init__(n_var=2, n_obj=1, n_constr=1, xl=0, xu=10)

    def _evaluate(self, x, out, *args, **kwargs):
        out["F"] = - np.min(x * [3, 1], axis=1)
        out["G"] = x[:, 0] + x[:, 1] - 10

In [17]:
import numpy as np

from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.factory import get_crossover, get_mutation, get_sampling
from pymoo.optimize import minimize

problem = MyProblem3()

algorithm = GA(
    pop_size=1,
    eliminate_duplicates=True,
)

res = minimize(
    problem,
    algorithm,
    ('n_gen', 200),
    seed=1,
    save_history=True
)

print("Best solution found: %s" % res.X)
print("Function value: %s" % res.F)
print("Constraint violation: %s" % res.CV)

Best solution found: [2.65683499 7.33397018]
Function value: [-7.33397018]
Constraint violation: [0.]


#### Example-4 multi objective .....pymoo 

In [30]:
demand_list = [12, 13, 15, 18]
supply_list = [15, 18, 10, 13]
class MyProblem(Problem):
    def __init__(self):
        super().__init__(n_var=len(pcs)*2,
                         n_obj=1,
                         n_constr=0,
                         xl=anp.array([-1]*len(pcs)*2),
                         xu=anp.array([2]*len(pcs)*2))

    def _evaluate(self, X, out, *args, **kwargs):
        cost = 0
        for x in X:
            var_length = len(x)//2
            balance_info = np.array(x[:var_length])
            cost = 0
            for balance,de,su in zip(balance_info, demand_list,supply_list):
                diff = su-de
                if su > de:
                    diff = su-de
                    cost += diff
                else:
                    cost -= diff**4
        out["F"] = np.column_stack([cost-1, cost+1])
        out["G"] = np.column_stack([cost-1, cost+1])

In [29]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.factory import get_problem
from pymoo.optimize import minimize

print("MyProblem.................................")
print(MyProblem())

# if we set pop size 1 then in f(x) x will be [[ n var ]] but if we increase pop size 2 then x will be [[ n var ],[ n var ] ]
# so we got in fitness function as per logic.
algorithm = NSGA2(
    pop_size=1,
    eliminate_duplicates=True)

import time
start_time = time.time()
res = minimize(MyProblem(),
               algorithm,
                ('n_gen', 100),
               seed=1,
               verbose=False)

print("--- %s seconds ---" % (time.time() - start_time))

print("Best solution found: %s" % res.X)

MyProblem.................................
# name: MyProblem
# n_var: 8
# n_obj: 1
# n_constr: 0

[[ 0.25106601  1.16097348 -0.99965688 -0.09300228 -0.55973233 -0.72298422
  -0.44121937  0.03668218]]
1
[[ 0.25106601  1.16097348 -0.99965688 -0.09300228 -0.91119948 -0.72298422
  -0.44121937  0.03668218]]
1
[[ 0.25106601  1.12195027 -0.99965688  0.01172751 -0.55973233 -0.61311961
  -0.44121937  0.03668218]]
1
[[ 0.25106601  1.16097348 -0.99965688 -0.09300228 -0.54273427 -0.72298422
  -0.44121937  0.03668218]]
1
[[ 0.25106601  1.16097348 -0.99965688 -0.32039241 -0.55973233 -0.72298422
  -0.44121937  0.03668218]]
1
[[ 0.25106601  1.16097348 -0.99968438 -0.32039241 -0.55973233 -0.70911991
  -0.44121937  0.03668218]]
1
[[ 0.25106601  1.16097348 -0.99971099 -0.25524656 -0.55973233 -0.72298422
  -0.52447316  0.03668218]]
1
[[ 0.28879126  1.16097348 -0.99965688 -0.32039241 -0.55973233 -0.72298422
  -0.44121937  0.03668218]]
1
[[ 0.25106601  1.16097348 -0.63901365 -0.32039241 -0.55973233 -0.72298

[[ 0.05510761  1.65939835 -0.99932171 -0.33872855 -0.26551299  0.06618124
  -0.00600853  1.3202916 ]]
1
[[ 0.05510761  1.65939835 -0.76323996 -0.33872855 -0.26551299  0.06618124
   0.01094342  1.3202916 ]]
1
[[ 0.05510761  1.65939835 -0.76323996 -0.33872855 -0.26551299  0.06618124
   0.01094342  1.42310177]]
1
[[ 0.05510761  1.66183623 -0.76323996 -0.33872855 -0.26551299  0.06618124
   0.01094342  1.42310177]]
1
[[ 0.05510761  1.53591394 -0.76323996 -0.33872855 -0.26551299  0.06618124
   0.01094342  1.42310177]]
1
[[ 0.05510761  1.66183623 -0.76323996 -0.33872855 -0.26551299  0.06618124
   0.16421712  1.42310177]]
1
[[ 0.05510761  1.66183623 -0.61831491 -0.33872855 -0.26551299  0.06618124
   0.16421712  1.42310177]]
1
[[-0.00302601  1.66183623 -0.61831491 -0.23239234 -0.26551299  0.06618124
   0.16421712  1.42310177]]
1
[[-0.00302601  1.66183623 -0.61831491 -0.23239234 -0.18528456  0.06618124
   0.16421712  1.42310177]]
1
[[-0.00302601  1.66183623 -0.61831491 -0.23239234 -0.38224153  0